In [79]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
from src.tradecli import *
from src.journal import *

In [81]:
# Generate the feature dataframe
full_df

,trade_uid,type_2_m15,confidence,type_2_d,entry_price,initial_risk_reward,type_3_m15,potential_return,management_strategy,type_1_M,...,side,type_2_h1,return,outcome,initial_return,type_1_m1,tp_price,htf_poi_ltf_confirmation,close_price,type_2_m5
0,1,None,NaN,None,1.1000,2.000000,True,NaN,no_management,None,...,long,None,NaN,None,0.0100,True,1.1100,None,NaN,None
1,2,None,NaN,None,1.2000,0.666667,None,NaN,no_management,None,...,long,True,-0.0145,loss,0.0100,None,1.2100,True,1.1855,None
2,3,None,NaN,None,1.3000,10.000000,None,NaN,no_management,None,...,long,None,NaN,None,0.0300,None,1.3300,False,NaN,None
3,4,None,4.0,None,1.3000,10.000000,None,0.0150,strategy_2,None,...,long,None,0.0050,win,0.0300,None,1.3300,False,1.3050,None
4,5,None,5.0,None,1.1378,5.000000,None,0.0299,no_management,None,...,long,True,0.0117,win,0.0285,True,1.1663,False,1.1495,None
5,6,None,2.0,True,1.1331,5.913580,None,0.0123,no_management,True,...,long,None,0.0180,win,0.0479,None,1.1810,False,1.1511,None
6,7,None,5.0,True,1.1161,7.482759,None,0.0491,be_after_push,None,...,long,None,NaN,None,0.0434,None,1.1595,False,NaN,None
7,8,None,3.0,True,1.1125,2.215190,None,0.0210,no_management,None,...,long,None,NaN,None,0.0175,None,1.1300,True,NaN,None
8,9,None,3.0,None,1.1495,3.978022,None,0.0232,close_early,True,...,long,None,-0.0176,loss,0.0362,None,1.1857,False,1.1319,True
9,10,True,5.0,None,1.1439,1.348315,None,0.0128,no_management,None,...,long,None,-0.0053,loss,0.0120,None,1.1559,True,1.1386,None


In [82]:
journal.to_markdown(output_dir='output')